<a href="https://colab.research.google.com/github/AhmadAlabrash/NLP/blob/main/NLP_gensim_%26_spacy_%26_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd


df = pd.read_csv("/content/Fake_Real_Data.csv")

In [2]:
df['label_num'] = df['label'].map({'Fake' : 0, 'Real': 1})

In [3]:
df

,Text,label,label_num
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,0
1,U.S. conservative leader optimistic of common ...,Real,1
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,1
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,0
4,Democrats say Trump agrees to work on immigrat...,Real,1
...,...,...,...
9895,Wikileaks Admits To Screwing Up IMMENSELY Wit...,Fake,0
9896,Trump consults Republican senators on Fed chie...,Real,1
9897,Trump lawyers say judge lacks jurisdiction for...,Real,1
9898,WATCH: Right-Wing Pastor Falsely Credits Trum...,Fake,0


In [5]:
import spacy
import spacy.cli
spacy.cli.download("en_core_web_lg")
nlp = spacy.load("en_core_web_lg")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [12]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

[=================================================-] 98.3% 1634.1/1662.8MB downloaded


In [ ]:
wv.get_vector('word')

In [ ]:
wv.get_vector('hi')

In [135]:
fter_process =[]
for word in nlp('hi man how are u today word amazonwashingtonpost trumpian trumpian trumpian'):

   if word.text not in wv.vocab :
     print('e')
   else :
     print(word)  

hi
man
how
are
u
today
word
e
e
e
e


In [128]:
import re
def clean(text):
   # letters_only = re.sub("[^a-zA-Z]",   " ",   str(text))
    text = re.sub(r'[^\w\s\']',' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip().lower()

In [129]:
df.Text =df.Text.apply(clean)

In [144]:
def check(text):
  fter_process =[]
  for word in nlp(text):

   if word.text not in wv.vocab :
     print('e')
   else :
     fter_process.append(word)  
   return ' '.join(str(v) for v in fter_process)  

In [ ]:
df['ch_text'] = df["Text"].apply(lambda x : check(x))

In [149]:
def preprocess(txt):
 after_process =[]
 x =0
 for word in nlp(txt):
    

        if  word.is_stop or word.is_punct or word.text == ' ' or word.is_alpha == False :
           x= x+1

        else :

           after_process.append(wv.get_vector(word.lemma_) )   


 
 return after_process

In [ ]:
df['vector'] = df['ch_text'].apply(lambda text: preprocess(text))

In [48]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    df.vector.values, 
    df.label_num, 
    test_size=0.2, 
)

In [50]:
import numpy as np

X_train_2d = np.stack(x_train)
X_test_2d =  np.stack(x_test)

In [53]:
X_train_2d

array([], shape=(7920, 0), dtype=float64)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

# creating a GradientBoosting model object
clf = GradientBoostingClassifier()

# fit with all_train_embeddings and y_train
clf.fit(X_train_2d, y_train)


# get the predictions for all_test_embeddings and store it in y_pred
y_pred = clf.predict(X_test_2d)


# print the classfication report
print(classification_report(y_test, y_pred))